### Config stuff

In [1]:
import os

import ConnectionConfig as cc
from delta import DeltaTable
cc.setupEnvironment()

Dynamically set JAVA_HOME: /Users/user/Library/Java/JavaVirtualMachines/temurin-21.0.2/Contents/Home


In [20]:
spark = cc.startLocalCluster("FACT_RIDE")
spark.getActiveSession()

# Fact transformations
This notebooks creates the sales fact table from scratch based on the operational source table "sales"
When creating a fact table always follow the listed steps in order.


#### 1 READ NECESSARY SOURCE TABLE(S) AND PERFORM TRANSFORMATIONS
**When reading from the source table make sure you include all data necessary:**
- to calculate the measure values
- the source table keys that you have to use to lookup the correct surrogate keys in the dimension tables.

**If more than one table is needed to gather the necesary information you can opt for one of two strategies:**
- Use a select query when reading from the jdbc source with the spark.read operation. Avoid complex queries because the operational database needs a lot of resources to run those queries.
- Perform a spark.read operation for each table separately and join the tables within Spark. The joins will take place on the cluster instead of the database. You limit the database recources used, but there can be a significant overhead of unnecessary data tranferred to the cluster.


In this case we just rename Amount and create a default count_mv column.
The transformations are minimal. In reality, transformations can be far more complex. If so, it can be advisable to work out the transforms in more then one step.*



In [21]:
cc.set_connectionProfile("default")
ride_src_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable","rides").option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "rideid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 1000) \
    .load()

subscriptions_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable","subscriptions").option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "subscriptionid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 1000) \
    .load()


# weather_df = spark.read.json(r'C:\Users\kkiva\data4_project_group5\examples\weather\*.json')


In [22]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, ArrayType

# Define the schema for the JSON structure based on the provided response
weather_schema = StructType([
    StructField("zipCode", StringType(), True),
    StructField("coord", StructType([
        StructField("lon", FloatType(), True),
        StructField("lat", FloatType(), True)
    ]), True),
    StructField("weather", ArrayType(StructType([
        StructField("id", IntegerType(), True),
        StructField("main", StringType(), True),
        StructField("description", StringType(), True),
        StructField("icon", StringType(), True)
    ])), True),
    StructField("base", StringType(), True),
    StructField("main", StructType([
        StructField("temp", FloatType(), True),
        StructField("feels_like", FloatType(), True),
        StructField("temp_min", FloatType(), True),
        StructField("temp_max", FloatType(), True),
        StructField("pressure", IntegerType(), True),
        StructField("humidity", IntegerType(), True),
        StructField("sea_level", IntegerType(), True),
        StructField("grnd_level", IntegerType(), True)
    ]), True),
    StructField("visibility", IntegerType(), True),
    StructField("wind", StructType([
        StructField("speed", FloatType(), True),
        StructField("deg", IntegerType(), True),
        StructField("gust", FloatType(), True)
    ]), True),
    StructField("rain", StructType([
        StructField("1h", FloatType(), True)
    ]), True),
    StructField("clouds", StructType([
        StructField("all", IntegerType(), True)
    ]), True),
    StructField("dt", IntegerType(), True),
    StructField("sys", StructType([
        StructField("type", IntegerType(), True),
        StructField("id", IntegerType(), True),
        StructField("country", StringType(), True),
        StructField("sunrise", IntegerType(), True),
        StructField("sunset", IntegerType(), True)
    ]), True),
    StructField("timezone", IntegerType(), True),
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("cod", IntegerType(), True)
])


AttributeError: 'StructType' object has no attribute 'createOrReplaceTempView'

In [23]:
weather_df = spark.read.option("multiline", "true").schema(weather_schema).json("weather/*.json")
weather_df.printSchema()
weather_df.show(5, truncate=False)


root
 |-- zipCode: string (nullable = true)
 |-- coord: struct (nullable = true)
 |    |-- lon: float (nullable = true)
 |    |-- lat: float (nullable = true)
 |-- weather: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- main: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- icon: string (nullable = true)
 |-- base: string (nullable = true)
 |-- main: struct (nullable = true)
 |    |-- temp: float (nullable = true)
 |    |-- feels_like: float (nullable = true)
 |    |-- temp_min: float (nullable = true)
 |    |-- temp_max: float (nullable = true)
 |    |-- pressure: integer (nullable = true)
 |    |-- humidity: integer (nullable = true)
 |    |-- sea_level: integer (nullable = true)
 |    |-- grnd_level: integer (nullable = true)
 |-- visibility: integer (nullable = true)
 |-- wind: struct (nullable = true)
 |    |-- speed: float (nullable = true)
 |    |-- de


#### 2 MAKE DIMENSION TABLES AVAILABLE AS VIEWS

In [25]:
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")
dim_vehicle = spark.read.format("delta").load("spark-warehouse/dimvehicle")
dim_user = spark.read.format("delta").load("spark-warehouse/dimuser")
dim_weather = spark.read.format("delta").load("spark-warehouse/dimweather")
dim_lock = spark.read.format("delta").load("spark-warehouse/dimlock")

dim_date.createOrReplaceTempView("dimDate")
dim_user.createOrReplaceTempView("dimUser")
dim_vehicle.createOrReplaceTempView("dimVehicle")
dim_weather.createOrReplaceTempView("dimWeather")
dim_lock.createOrReplaceTempView("dimLock")


#### 3 Build the fact table

Within the creation of a fact table always perform these two tasks:
1.   Include the measures of the fact
2. Use the dimension tables to look up the surrogate keys that correspond with the natural key value. In case of SCD2 dimension use the scd_start en scd_end to find the correct version of the data in the dimension


In [26]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import math

def haversine_km(lat1, lon1, lat2, lon2):
    if None in (lat1, lon1, lat2, lon2):  # Handle NULL values safely
        return None  
    R = 6371  # Radius of Earth in km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Register the UDF in Spark SQL
haversine_udf = udf(haversine_km, DoubleType())
spark.udf.register("haversine_km", haversine_udf)


In [31]:
ride_src_df.createOrReplaceTempView("rides_source")
subscriptions_df.createOrReplaceTempView("subscriptions_source")

# Create temp views for the required dataframes
weather_df.createOrReplaceTempView("weather_data")

ridesFactFromSource = spark.sql("""
    SELECT src.rideid AS ride_id, 
           du.userSK AS user_sk, 
           src.startlockid AS start_lock_id, 
           src.endlockid AS end_lock_id, 
           dd.date_sk AS date_sk, 
           dv.vehicleid AS vehicle_id, 
           (src.endtime - src.starttime) AS ride_duration, 
           haversine_km(
               CAST(SPLIT(REPLACE(dl_start.gpscoord, '(', ''), ',')[0] AS DOUBLE),
               CAST(SPLIT(REPLACE(REPLACE(dl_start.gpscoord, ')', ''), '(', ''), ',')[1] AS DOUBLE),
               CAST(SPLIT(REPLACE(dl_end.gpscoord, '(', ''), ',')[0] AS DOUBLE),
               CAST(SPLIT(REPLACE(REPLACE(dl_end.gpscoord, ')', ''), '(', ''), ',')[1] AS DOUBLE)
           ) AS distance_km, dw.weather_id,  
           md5(concat(src.rideid, du.userSK, src.startlockid, src.endlockid, dd.date_sk, dv.vehicleid)) AS md5 
    FROM rides_source AS src 
    LEFT OUTER JOIN subscriptions_source AS sub ON src.subscriptionid = sub.subscriptionid 
    LEFT OUTER JOIN dimUser AS du ON sub.userid = du.userSK 
    LEFT OUTER JOIN dimLock AS dl_start ON src.startlockid = dl_start.lockid  
    LEFT OUTER JOIN dimLock AS dl_end ON src.endlockid = dl_end.lockid  
    LEFT OUTER JOIN dimDate AS dd ON DATE(src.starttime) = dd.date 
    LEFT OUTER JOIN dimVehicle AS dv ON src.vehicleid = dv.vehicleid 
    LEFT OUTER JOIN weather_data AS wd 
        ON dl_start.zipcode = wd.zipcode  -- Match postal code (from dimLock) with weather data
        AND date_trunc('hour', src.starttime) = date_trunc('hour', wd.weather_time)  -- Match the rounded hour of the trip's start time with weather data time
    LEFT OUTER JOIN dimWeather AS dw 
        ON wd.weather_condition = dw.weather_condition  -- Match weather condition from JSON to weather dimension
""")


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `wd`.`weather_time` cannot be resolved. Did you mean one of the following? [`wd`.`weather`, `dd`.`date_sk`, `src`.`endtime`, `du`.`userid`, `wd`.`base`].; line 25 pos 67;
'Project ['src.rideid AS ride_id#2867, 'du.userSK AS user_sk#2868, 'src.startlockid AS start_lock_id#2869, 'src.endlockid AS end_lock_id#2870, 'dd.date_sk AS date_sk#2871, 'dv.vehicleid AS vehicle_id#2872, ('src.endtime - 'src.starttime) AS ride_duration#2873, 'haversine_km(cast('SPLIT('REPLACE('dl_start.gpscoord, (, ), ,)[0] as double), cast('SPLIT('REPLACE('REPLACE('dl_start.gpscoord, ), ), (, ), ,)[1] as double), cast('SPLIT('REPLACE('dl_end.gpscoord, (, ), ,)[0] as double), cast('SPLIT('REPLACE('REPLACE('dl_end.gpscoord, ), ), (, ), ,)[1] as double)) AS distance_km#2874, 'dw.weather_id, 'md5('concat('src.rideid, 'du.userSK, 'src.startlockid, 'src.endlockid, 'dd.date_sk, 'dv.vehicleid)) AS md5#2875]
+- 'Join LeftOuter, (

In [28]:
ride_src_df.createOrReplaceTempView("rides_source")
subscriptions_df.createOrReplaceTempView("subscriptions_source")

In [41]:
# trying to fix null subscription ids 
xd = spark.sql("""
SELECT src.subscriptionid 
FROM rides_source AS src
LEFT JOIN subscriptions_source AS sub ON sub.subscriptionid = src.subscriptionid
WHERE sub.subscriptionid IS NULL;

""")
xd.show()

+--------------+
|subscriptionid|
+--------------+
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
+--------------+
only showing top 20 rows



In [29]:
ridesFactFromSource.printSchema()
ridesFactFromSource.show(20)

root
 |-- ride_id: integer (nullable = true)
 |-- user_sk: string (nullable = true)
 |-- start_lock_id: integer (nullable = true)
 |-- end_lock_id: integer (nullable = true)
 |-- date_sk: long (nullable = true)
 |-- vehicle_id: integer (nullable = true)
 |-- ride_duration: interval day to second (nullable = true)
 |-- distance_km: double (nullable = true)
 |-- md5: string (nullable = true)



25/03/29 17:42:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------+-------------+-----------+-------+----------+--------------------+------------------+----+
|ride_id|user_sk|start_lock_id|end_lock_id|date_sk|vehicle_id|       ride_duration|       distance_km| md5|
+-------+-------+-------------+-----------+-------+----------+--------------------+------------------+----+
|      3|   NULL|         2046|       1951|   2455|      NULL|INTERVAL '-1095 0...|1.4949081239108692|NULL|
|     11|   NULL|          985|       2148|   2455|      NULL|INTERVAL '-1095 0...| 2.067612187914813|NULL|
|     13|   NULL|         5619|       2717|   2455|      NULL|INTERVAL '-1095 0...| 5.381957169382341|NULL|
|     17|   NULL|         2046|       1951|   3916|      NULL|INTERVAL '0 00:02...|1.4949081239108692|NULL|
|      1|   NULL|         4849|       3188|   2455|      NULL|INTERVAL '-1095 0...| 3.491018931475966|NULL|
|      4|   NULL|         1821|       2186|   2455|      NULL|INTERVAL '-1095 0...|2.3114308809290756|NULL|
|      5|   NULL|         63

## Initial load
The first time loading the fact table perform a FULL load. All data is written to the Delta Table.
After initial load the code line has to be disabled

In [6]:

ridesFactFromSource.write.format("delta").mode("overwrite").saveAsTable("factRides")



## Incremental load
When previous runs where performend you can opt for a 'faster' incremental run that only writes away changes. UPDATES and INSERTS are performed in one run.
In our solution we use an md5 based on all fields in the source table to detect changes. This is not the most efficient way to detect changes. A better way is to use a timestamp field in the source table and use that to detect changes. This is not implemented in this example.

In [7]:
dt_factRides = DeltaTable.forPath(spark,".\spark-warehouse\\factrides")
dt_factRides.toDF().createOrReplaceTempView("factRides_current")
#Merge to perform updates (TODO: Implement md5 strategy)

result = spark.sql("MERGE INTO factRides_current AS target \
      using factRides_new AS source ON target.rideID = source.rideID \
      WHEN MATCHED and source.MD5<>target.MD5 THEN UPDATE SET * \
      WHEN NOT MATCHED THEN INSERT *")

result.show()

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                0|               0|               0|                0|
+-----------------+----------------+----------------+-----------------+


In [8]:
# IMPORTANT: ALWAYS TEST THE CREATED CODE.
# In this example I changed order 498 in the operational database and checked the change after the run.
# spark.sql("select * from factsales f join dimsalesrep ds on f.salesrepSK = ds.salesrepSK where OrderID = 192  ").show()
spark.sql("select count(*) from factrides").show()
spark.sql("select * from factrides where rideId=1").show()



+--------+
|count(1)|
+--------+
|     999|
+--------+

+-------+------+--------------------+--------+----------+--------------------+
|OrderID|dateSK|          salesrepSK|count_mv|revenue_mv|                 md5|
+-------+------+--------------------+--------+----------+--------------------+
|      1|   650|b65df3d9-20dc-42d...|       1| 851804379|a237b06f2932af7dd...|
+-------+------+--------------------+--------+----------+--------------------+


### Checking the history of your delta fact table

In [9]:
# The history information is derived from the delta table log files. They contain a lot of information of all the actions performed on the table. In this case it tells us something about de merge operations. You can find statistics about the update and insert counts in the document.

fact.history().show(10,False)

NameError: name 'fact' is not defined

In [13]:
spark.stop()